In [1]:
import pandas as pd
import numpy as np 
import matplotlib as mpl
import seaborn as sns

In [2]:
FEATURE = "Unemployment"
df = pd.read_csv("..\..\datasets\Original\%s.csv" % FEATURE)
df

,Series Name,Series Code,Country Name,Country Code,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,Afghanistan,AFG,11.15799999,11.18000031,11.15200043,11.21700001,11.71000004
1,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,Albania,ALB,15.42000008,13.61999989,12.30000019,11.47000027,13.32900047
2,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,Algeria,DZA,10.19999981,10.33300018,10.42000008,10.51299953,12.55000019
3,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,American Samoa,ASM,..,..,..,..,..
4,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,Andorra,AND,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...
212,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,Virgin Islands (U.S.),VIR,12.67700005,12.32499981,12.42599964,12.46800041,13.54500008
213,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,West Bank and Gaza,PSE,23.94000053,25.68000031,26.26000023,25.34000015,25.88999939
214,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,"Yemen, Rep.",YEM,13.43299961,13.29699993,13.14500046,13.05599976,13.39099979
215,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,Zambia,ZMB,10.87199974,11.63000011,12.01000023,12.52000046,12.84799957


## Skeleton dataframe

In [3]:
columns = ["Country Code"]
df.drop_duplicates(subset=columns, keep="first")
df_new = df[columns[0]].unique()
df_new = pd.DataFrame(df_new, columns=columns)
df.rename(columns={"Country Code": "ISO3_Code"}, inplace=True)
df_new.rename(columns={"Country Code": "ISO3_Code"}, inplace=True)
df_new.shape

(217, 1)

In [4]:
df_code = pd.read_csv("../../datasets/Original/AreaCode_ISO3Code_conversion.csv")
df_code = df_code[["Country", "ISO3 Code", "M49 Code"]]
df_new = df_new.join(df_code.set_index("ISO3 Code"), on="ISO3_Code", validate="1:1")
df_new

,ISO3_Code,Country,M49 Code
0,AFG,Afghanistan,4.0
1,ALB,Albania,8.0
2,DZA,Algeria,12.0
3,ASM,American Samoa,16.0
4,AND,Andorra,20.0
...,...,...,...
212,VIR,United States Virgin Islands,850.0
213,PSE,Palestine,275.0
214,YEM,Yemen,887.0
215,ZMB,Zambia,894.0


In [5]:
df_new.rename(columns={"ISO3 Code": "ISO3_Code", "Country": "Country_Name", "M49 Code": "M49_Code"}, inplace=True)
df_new.head()

,ISO3_Code,Country_Name,M49_Code
0,AFG,Afghanistan,4.0
1,ALB,Albania,8.0
2,DZA,Algeria,12.0
3,ASM,American Samoa,16.0
4,AND,Andorra,20.0


In [6]:
df_new[df_new["Country_Name"].isnull()]

,ISO3_Code,Country_Name,M49_Code
39,CHI,NaN,NaN
105,XKX,NaN,NaN


In [7]:
# drop the null rows cause it's the legend
df_new.dropna(inplace=True)
df_new

,ISO3_Code,Country_Name,M49_Code
0,AFG,Afghanistan,4.0
1,ALB,Albania,8.0
2,DZA,Algeria,12.0
3,ASM,American Samoa,16.0
4,AND,Andorra,20.0
...,...,...,...
212,VIR,United States Virgin Islands,850.0
213,PSE,Palestine,275.0
214,YEM,Yemen,887.0
215,ZMB,Zambia,894.0


## Unemployment
Unemployment (% of labor force)

In [8]:
df_tmp = pd.concat([df.loc[:, ["ISO3_Code"]], df.iloc[:, -5:]], axis=1)
df_new = df_new.merge(df_tmp, on="ISO3_Code", how="inner")
columns = df.columns[-5:].values
names_dict = {}
for i, column in enumerate(columns):
    names_dict[column] = "%s_%s" % (FEATURE, column[:4])
df_new.rename(columns=names_dict, inplace=True)
df_new

,ISO3_Code,Country_Name,M49_Code,Unemployment_2016,Unemployment_2017,Unemployment_2018,Unemployment_2019,Unemployment_2020
0,AFG,Afghanistan,4.0,11.15799999,11.18000031,11.15200043,11.21700001,11.71000004
1,ALB,Albania,8.0,15.42000008,13.61999989,12.30000019,11.47000027,13.32900047
2,DZA,Algeria,12.0,10.19999981,10.33300018,10.42000008,10.51299953,12.55000019
3,ASM,American Samoa,16.0,..,..,..,..,..
4,AND,Andorra,20.0,..,..,..,..,..
...,...,...,...,...,...,...,...,...
210,VIR,United States Virgin Islands,850.0,12.67700005,12.32499981,12.42599964,12.46800041,13.54500008
211,PSE,Palestine,275.0,23.94000053,25.68000031,26.26000023,25.34000015,25.88999939
212,YEM,Yemen,887.0,13.43299961,13.29699993,13.14500046,13.05599976,13.39099979
213,ZMB,Zambia,894.0,10.87199974,11.63000011,12.01000023,12.52000046,12.84799957


In [9]:
# add a column for average over the years
columns = df_new.columns[-5:]
df_cols = df_new[columns].applymap(lambda x: float(x) if x not in ["", None, ".."] else None)
df_new = pd.concat([df_new.iloc[:, :3], df_cols], axis=1)
df_new["%s_Avg" % FEATURE] = df_new[columns].mean(axis=1)
df_new.head()

,ISO3_Code,Country_Name,M49_Code,Unemployment_2016,Unemployment_2017,Unemployment_2018,Unemployment_2019,Unemployment_2020,Unemployment_Avg
0,AFG,Afghanistan,4.0,11.158,11.180,11.152,11.217,11.710,11.2834
1,ALB,Albania,8.0,15.420,13.620,12.300,11.470,13.329,13.2278
2,DZA,Algeria,12.0,10.200,10.333,10.420,10.513,12.550,10.8032
3,ASM,American Samoa,16.0,NaN,NaN,NaN,NaN,NaN,NaN
4,AND,Andorra,20.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# prettify the float
#df_new = df_new[df_new.columns[-6:]].applymap(na_action='ignore', func=lambda x: ".6f" % x)
with open("../../datasets/Processed/%s_Processed.csv" % FEATURE, "w", encoding="utf8") as f:
    df_new.to_csv(f, lineterminator="\n", index=False)

## Stats

In [11]:
df_new.isnull().sum()

ISO3_Code             0
Country_Name          0
M49_Code              0
Unemployment_2016    29
Unemployment_2017    29
Unemployment_2018    29
Unemployment_2019    29
Unemployment_2020    29
Unemployment_Avg     29
dtype: int64